In [1]:
import cv2
import os
import PIL
import tensorflow_hub as hub

In [2]:
data_dir =r"F:\project\COVID-19_Radiography_Dataset\data"

In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('F:/project/COVID-19_Radiography_Dataset/data')

In [4]:
import random 
COVID = list(data_dir.glob('COVID/*'))
COVID = random.sample(COVID,1000)
len(COVID)

1000

In [5]:
Lung_Opacity= list(data_dir.glob('Lung_Opacity/*'))
Lung_Opacity = random.sample(Lung_Opacity,1000)
len(Lung_Opacity)

1000

In [6]:
Normal = list(data_dir.glob('Normal/*'))
Normal = random.sample(Normal,1000)
len(Normal)

1000

In [7]:
Viral_Pneumonia = list(data_dir.glob('Viral Pneumonia/*'))
Viral_Pneumonia = random.sample(Viral_Pneumonia,1000)
len(Viral_Pneumonia)

1000

In [8]:
images_dict = {
    'COVID': COVID,
    'Lung_Opacity': Lung_Opacity,
    'Normal': Normal,
    'Viral_Pneumonia': Viral_Pneumonia,
    
}

In [9]:
labels_dict = {
    'COVID': 0,
    'Lung_Opacity':1,
    'Normal': 2,
    'Viral_Pneumonia': 3,
    
}

In [10]:
import numpy as np
X, y = [], []

for label, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        #resized_img=resized_img.astype(np.uint8)
        #resized_img=resized_img/np.max(resized_img)
        X.append(resized_img)
        y.append(labels_dict[label])

In [11]:
import numpy 
X = numpy.array(X)
y = numpy.array(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [13]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [14]:
feature_extractor_model = "https://tfhub.dev/tensorflow/resnet_50/classification/1"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [15]:
import tensorflow as tf
num_of_flowers = 4

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              25612201  
_________________________________________________________________
dense (Dense)                (None, 4)                 4008      
Total params: 25,616,209
Trainable params: 4,008
Non-trainable params: 25,612,201
_________________________________________________________________


In [16]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
100/100 [==============================] - 252s 2s/step - loss: 1.3743 - acc: 0.4547
Epoch 2/5
100/100 [==============================] - 231s 2s/step - loss: 1.3453 - acc: 0.6428
Epoch 3/5
100/100 [==============================] - 234s 2s/step - loss: 1.3179 - acc: 0.6634
Epoch 4/5
100/100 [==============================] - 236s 2s/step - loss: 1.2919 - acc: 0.6525
Epoch 5/5
100/100 [==============================] - 232s 2s/step - loss: 1.2672 - acc: 0.6556


In [17]:
model.evaluate(X_test_scaled,y_test)

25/25 [==============================] - 58s 2s/step - loss: 1.2541 - acc: 0.6737


[1.254119634628296, 0.6737499833106995]

In [18]:
model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
100/100 [==============================] - 233s 2s/step - loss: 1.2437 - acc: 0.6622
Epoch 2/10
100/100 [==============================] - 223s 2s/step - loss: 1.2214 - acc: 0.6681
Epoch 3/10
100/100 [==============================] - 210s 2s/step - loss: 1.2002 - acc: 0.6750
Epoch 4/10
100/100 [==============================] - 212s 2s/step - loss: 1.1801 - acc: 0.6819
Epoch 5/10
100/100 [==============================] - 211s 2s/step - loss: 1.1609 - acc: 0.6756
Epoch 6/10
100/100 [==============================] - 211s 2s/step - loss: 1.1428 - acc: 0.6806
Epoch 7/10
100/100 [==============================] - 216s 2s/step - loss: 1.1256 - acc: 0.6847
Epoch 8/10
100/100 [==============================] - 215s 2s/step - loss: 1.1094 - acc: 0.6834
Epoch 9/10
100/100 [==============================] - 209s 2s/step - loss: 1.0939 - acc: 0.6837
Epoch 10/10
100/100 [==============================] - 219s 2s/step - loss: 1.0792 - acc: 0.6878


In [19]:
model.evaluate(X_test_scaled,y_test)

25/25 [==============================] - 50s 2s/step - loss: 1.0673 - acc: 0.7013


[1.067314863204956, 0.7012500166893005]